In [1]:
# ---------------------------------------------------------------------------- #
# An implementation of https://arxiv.org/pdf/1512.03385.pdf                    #
# See section 4.2 for the model architecture on CIFAR-10                       #
# Some part of the code was referenced from below                              #
# https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py   #
# ---------------------------------------------------------------------------- #

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

In [2]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4), #Pad the given PIL Image on all sides with the given "pad" value.
    transforms.RandomHorizontalFlip(), #Horizontally flip the given PIL Image randomly with a given probability.
    transforms.RandomCrop(32), #Crop the given PIL Image at a random location. transforms.RandomCrop(size): desired output size
    transforms.ToTensor()]) #Convert a ``PIL Image`` or ``numpy.ndarray`` to tensor.

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

Files already downloaded and verified


In [3]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    # it won't change width and height when stride=1
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        #self.downsample determine this block be identity block or conv block
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        #add to shortcut, and then go into activation function
        out = self.relu(out)
        return out

In [4]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16 #record the current layer's channel, will be used by next layer
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        #block being the ResidualBlock defined above
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        # here downsample means the (H,W) be changed(stride!=1)
        #  or the channels be changed (self.in_channels != out_channels)
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        # first block being conv block
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels #record the current layer's channel, will be used by next layer
        #layers already has 1 block, here append (blocks-1) more blocks
        # remaining blocks being identity block
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers) #nn.Sequential(*args), it don't accept list!
    
    def forward(self, x):
        out = self.conv(x) #input: (100,3,32,32)
        out = self.bn(out) #input: (100,16,32,32)
        out = self.relu(out)
        out = self.layer1(out) #input: (100,16,32,32), out_channels=16, blocks=2, stride=1
        out = self.layer2(out) #input: (100,16,32,32), out_channels=32, blocks=2, stride=2
        out = self.layer3(out) #input: (100,32,16,16), out_channels=64, blocks=2, stride=2
        out = self.avg_pool(out) #input: (100,64,8,8), average over the range of (8,8)
        out = out.view(out.size(0), -1) #input: (100,64,1,1)
        out = self.fc(out) #input: (100,64), output: (100,10)
        return out
    
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

In [5]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train the model
# train_loader: torch.utils.data.dataloader.DataLoader
# list(train_loader): it takes time for this conversion
# train_loader[0]: TypeError: 'DataLoader' object does not support indexing
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item())) #Tensor.item(): get the number

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.6381
Epoch [1/80], Step [200/500] Loss: 1.4063
Epoch [1/80], Step [300/500] Loss: 1.3042
Epoch [1/80], Step [400/500] Loss: 1.3419
Epoch [1/80], Step [500/500] Loss: 1.2542
Epoch [2/80], Step [100/500] Loss: 0.9422
Epoch [2/80], Step [200/500] Loss: 1.0328
Epoch [2/80], Step [300/500] Loss: 0.9152
Epoch [2/80], Step [400/500] Loss: 1.0899
Epoch [2/80], Step [500/500] Loss: 1.0359
Epoch [3/80], Step [100/500] Loss: 0.7884
Epoch [3/80], Step [200/500] Loss: 0.9706
Epoch [3/80], Step [300/500] Loss: 0.8849
Epoch [3/80], Step [400/500] Loss: 0.9697
Epoch [3/80], Step [500/500] Loss: 1.0077
Epoch [4/80], Step [100/500] Loss: 0.6495
Epoch [4/80], Step [200/500] Loss: 0.8861
Epoch [4/80], Step [300/500] Loss: 0.7222
Epoch [4/80], Step [400/500] Loss: 0.7949
Epoch [4/80], Step [500/500] Loss: 0.6071
Epoch [5/80], Step [100/500] Loss: 0.8682
Epoch [5/80], Step [200/500] Loss: 0.8192
Epoch [5/80], Step [300/500] Loss: 0.8283
Epoch [5/80], Step [400/500] Loss:

Epoch [39/80], Step [300/500] Loss: 0.3084
Epoch [39/80], Step [400/500] Loss: 0.2518
Epoch [39/80], Step [500/500] Loss: 0.3097
Epoch [40/80], Step [100/500] Loss: 0.1616
Epoch [40/80], Step [200/500] Loss: 0.2380
Epoch [40/80], Step [300/500] Loss: 0.3345
Epoch [40/80], Step [400/500] Loss: 0.2681
Epoch [40/80], Step [500/500] Loss: 0.2618
Epoch [41/80], Step [100/500] Loss: 0.3043
Epoch [41/80], Step [200/500] Loss: 0.2262
Epoch [41/80], Step [300/500] Loss: 0.3788
Epoch [41/80], Step [400/500] Loss: 0.1654
Epoch [41/80], Step [500/500] Loss: 0.1029
Epoch [42/80], Step [100/500] Loss: 0.2990
Epoch [42/80], Step [200/500] Loss: 0.1903
Epoch [42/80], Step [300/500] Loss: 0.1478
Epoch [42/80], Step [400/500] Loss: 0.2080
Epoch [42/80], Step [500/500] Loss: 0.1920
Epoch [43/80], Step [100/500] Loss: 0.2033
Epoch [43/80], Step [200/500] Loss: 0.1397
Epoch [43/80], Step [300/500] Loss: 0.1790
Epoch [43/80], Step [400/500] Loss: 0.1865
Epoch [43/80], Step [500/500] Loss: 0.1984
Epoch [44/8

Epoch [77/80], Step [400/500] Loss: 0.1835
Epoch [77/80], Step [500/500] Loss: 0.1643
Epoch [78/80], Step [100/500] Loss: 0.1166
Epoch [78/80], Step [200/500] Loss: 0.1614
Epoch [78/80], Step [300/500] Loss: 0.1336
Epoch [78/80], Step [400/500] Loss: 0.1137
Epoch [78/80], Step [500/500] Loss: 0.1340
Epoch [79/80], Step [100/500] Loss: 0.1601
Epoch [79/80], Step [200/500] Loss: 0.1393
Epoch [79/80], Step [300/500] Loss: 0.1718
Epoch [79/80], Step [400/500] Loss: 0.2374
Epoch [79/80], Step [500/500] Loss: 0.1827
Epoch [80/80], Step [100/500] Loss: 0.0947
Epoch [80/80], Step [200/500] Loss: 0.1177
Epoch [80/80], Step [300/500] Loss: 0.1424
Epoch [80/80], Step [400/500] Loss: 0.1154
Epoch [80/80], Step [500/500] Loss: 0.1167


In [7]:
# Test the model
model.eval() #Sets the module in evaluation mode.
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')

Accuracy of the model on the test images: 88.2 %
